# pyWECcast - v0.2.2
## WEC Powerseries Generator

This version release represents a refactoring to reduce dependence on specific file configurations for powerseries generation.

This is also the first implementation of numba accelerated functions with parallel functionality included

The example shows what may be a basic example of combining multiple sea-states


In [1]:
# General import directly from the local pyWECcast directory

import sys
sys.path.insert(1, '../')

import src as wc

In [2]:
baseDir = f'/mnt/c/Users/abharath/Documents/Projects/Grid_Value'
dataDir = f'{baseDir}/data'
buoyDB = f'{dataDir}/buoy_downloads.h5'
resultDB = f'{dataDir}/testing_buoy_powerseries.h5'

In [3]:
# In this example we will combined both wind and swell waves to estimate the WEC power output 
# based on a fully realised sea

buoyNum = '46041'

time_index, SwH, SwP, SwD = wc.extract_NOAA_buoy(buoyDB,buoyNum,'SwH', 'SwP', 'SwD')
time_index, WWH, WWP, WWD = wc.extract_NOAA_buoy(buoyDB,buoyNum,'WWH', 'WWP', 'WWD')


In [4]:
# for the particular NOAA buoy data files we have convenience functions in pyWECcast.ufuncs 
# to convert to a numerical format.

timestamps = wc.ufuncs.binary_timestamp_convert(time_index)

swell_D = wc.ufuncs.binary_direction_convert(SwD)
wind_D = wc.ufuncs.binary_direction_convert(WWD)


../src/ufuncs/utility_functions.py:77: UserWarning: Directional ndarray contains NaNs - default value inserted 0.0
  warnings.warn(warn,UserWarning)


In [5]:
# we can manipulate the data to reduuce computation. Note array shapes must remain equal

cut = 10
timestamps, SwH, SwP, SwD, WWH, WWP, WWD = (timestamps[:cut], SwH[:cut], SwP[:cut], swell_D[:cut], WWH[:cut], 
                                            WWP[:cut], wind_D[:cut])

In [6]:
# With a WECSim powerseries matrix, we can link the sea-state data to the WEC simulations 
# that are available. The closest match can be determined using a nearest neighbor method on 
# separate Hs, Tp and direction values or combinations of the groups

# Here we create links for both the wind and swell waves

wecSim = f'{dataDir}/WECSim_dataset_RM3_scale_0-1873.hdf5'

swell, measured = wc.link_sea_states(wecSim,SwH,SwP)
wind, measured = wc.link_sea_states(wecSim,WWH,WWP)


In [7]:
# With the sea-states in the dataset linked to what is available in the WECSim simulation db
# We can calculate the FFTs to obtian the reconstruction coefficients for both the wind and 
# swell waves

# if inMemory:True, calculate_fft_matrix() will return coefficients as a dict

swell_matrix = wc.calculate_fft_matrix(wecSim,swell[:,0],swell[:,1],inMemory=True)
wind_matrix = wc.calculate_fft_matrix(wecSim,wind[:,0],wind[:,1],inMemory=True)


In [8]:
# if we are looking to simulate multiple WECs, there is a convinience function in place to
# setup the fft_matrix to do this.

ndeg = 18

swell_matrix = wc.multiple_realizations(ndeg,swell_matrix)
wind_matrix = wc.multiple_realizations(ndeg,wind_matrix)


In [9]:
# with the general fft coefficient matrices created we now look to scale the amplitudes based
# on their incident direction to the WEC.

swell_matrix = wc.wec_direction(swell_matrix,ang='deg')
wind_matrix = wc.wec_direction(wind_matrix,ang='deg')
        

In [10]:
# in an example case using a potential WEC power acceptance window (here estimated to be gaussian for RM3)
# we can apply that windowing to the FFTs based on a window center and window function

from scipy.signal import gaussian

swell_matrix = wc.wec_window(swell_matrix,gaussian,center=90,ang='deg',std=1)
wind_matrix = wc.wec_window(wind_matrix,gaussian,center=90,ang='deg',std=1)

In [11]:
# Based on cosine sea-state spreading we can determine the interaction between the direction of the incident waves
# and the WEC acceptance window in teh reconstruction by first building a swell window matrix

swell_window = wc.wave_window(swell_matrix,SwD,Omax=15,ang='deg')
wind_window = wc.wave_window(wind_matrix,WWD,Omax=15,ang='deg')

In [12]:
# With the scaled coefficients and frequencies we can not reconstruct the powerseries for a single WEC with
# acceptance window and wave directionality taken into account

# In this case we require that inPhase=True is selected so that the WEC will be considered at the same point
# accross all directions

freq = '60S'

swell_result, time = wc.construct_powerseries(timestamps,freq,swell[:,0],swell[:,1],swell_window,
                         fft_matrix=swell_matrix,inMemory=True,inPhase=True)
wind_result, time = wc.construct_powerseries(timestamps,freq,wind[:,0],wind[:,1],wind_window,
                         fft_matrix=wind_matrix,inMemory=True,inPhase=True)

100%|██████████| 9/9 [00:06<00:00,  1.48it/s]


In [13]:
# to Naively combine power output from both wind and swell wave seas we simply sum the power
# generated from each individual sea

from numpy import sum, dstack

combination = dstack([swell_result,wind_result])
sumResults = sum(combination,axis=-1)

In [14]:
# Plots of the results can be generated to view the wind, swell and combine power outputs along
# with the means between any number of WECs

%matplotlib widget

import matplotlib.pyplot as plt
from numpy import mean

plt.plot(time,sum(swell_result,axis=-1),label='Swell along')
    
plt.plot(time,sum(wind_result,axis=-1),label='Wind along')

plt.plot(time,sum(sumResults,axis=-1),label='Sum of wind and swell')
    
plt.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …